In [2]:
!ls /mnt/chicm/data/retrieval/vectors

D_tmp.npy						 founds_tmp.npy
FeatureNet_se_resnext50_32x4d.index_retrieval		 global_noresize
FeatureNetV2_se_resnext50_32x4d.index_retrieval_2	 I_tmp.npy
FeatureNetV2_se_resnext50_32x4d.index_retrieval_2_local  local


In [8]:
VECTOR_DIR = '/mnt/chicm/data/retrieval/vectors'

In [6]:
!ls /mnt/chicm/data/retrieval

data_check.py
google_landmark_boxes_split1.csv
google_landmark_boxes_split2.csv
google-landmarks-dataset.zip
index_clean.csv
index.csv
index_imgs
index_imgs_256_crop
index_imgs_256_nocrop
landmark-retrieval-challenge-image-downloader.py
models
retrieval_sample_submission.csv
retrieval_sample_submission.csv.zip
test.csv
train_clean.csv
train_clean_invert.csv
train.csv
train_imgs
train_imgs_256_crop
vectors


In [3]:
!ls

balanced_loader.py   post_process.ipynb		  train_cls.py
data_analysis.ipynb  __pycache__		  train_retrieval.py
ensemble.py	     settings.py		  triplet_loss.py
loader.py	     settings_retrieval.py	  Untitled.ipynb
loader_retrieval.py  sub_retrieval_global.csv	  vec_pred.py
models.py	     sub_retrieval_local_224.csv  vector_search.ipynb
net		     sub_retrieval_local.csv
output		     test_faiss.py


In [9]:
import pandas as pd
import numpy as np
import os

In [5]:
df = pd.read_csv('sub_retrieval_local_224.csv')
df.head()

,id,images
0,cb9998b8cdaf6235,8fbdf0fdc15d43d0 c13f8c34eed9bf03 861d88c5618b...
1,30728cf6e50a6bc6,554d72a10542c954 35c429a5639bd122 86d4b7c9bd0d...
2,16afbc86b710337d,6e614e272d1c596b 6ef3e2793082f236 16048dcf423c...
3,d29b2166cf522450,5bd2269c9fa1836b f1c445e4f7528f3c 953aafe3c53d...
4,dd5c03b20c21cfba,ec477e789504ebec 7c3b3512c332ffe6 e910b3a94e88...


In [14]:
D = np.load(os.path.join(VECTOR_DIR,'D_tmp.npy'))
I = np.load(os.path.join(VECTOR_DIR,'I_tmp.npy'))
founds = np.load(os.path.join(VECTOR_DIR,'founds_tmp.npy'))

In [16]:
print(D.shape, I.shape, founds.shape)

(117703, 100) (117703, 100) (117703,)


In [21]:
founds.sum()

113654

In [44]:
D[0]

array([0.5560349 , 0.62239635, 0.6441932 , 0.6630283 , 0.66942537,
       0.6751373 , 0.67855716, 0.7255    , 0.73224676, 0.732486  ,
       0.7419702 , 0.7430555 , 0.74581975, 0.74780256, 0.7550701 ,
       0.758991  , 0.7608342 , 0.770538  , 0.77942264, 0.78043157,
       0.79701835, 0.8007183 , 0.802832  , 0.80776733, 0.80864793,
       0.8123764 , 0.81704044, 0.81935537, 0.8216411 , 0.8221303 ,
       0.82614106, 0.82776976, 0.82814646, 0.82907665, 0.83221215,
       0.8329355 , 0.8338907 , 0.84345025, 0.851492  , 0.86423266,
       0.86433774, 0.8643615 , 0.8657383 , 0.8662136 , 0.86909103,
       0.87063557, 0.8708782 , 0.87089515, 0.8717643 , 0.8740746 ,
       0.8743501 , 0.8767405 , 0.87676644, 0.87987494, 0.88065964,
       0.8813366 , 0.8824966 , 0.88353753, 0.88410604, 0.8855665 ,
       0.8864686 , 0.8866177 , 0.8909532 , 0.8920526 , 0.89226973,
       0.8973112 , 0.8978267 , 0.8981155 , 0.8982519 , 0.90100837,
       0.9013191 , 0.9059922 , 0.9109571 , 0.9151891 , 0.91592

In [47]:
(D < 0.95).astype(np.int).sum() / (founds.sum()*100)

0.7984033118060077

In [54]:
mask = (D < 0.95).astype(np.int) * 2 -1
mask

array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       ...,
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ..., -1, -1, -1]])

In [55]:
maskedI = I * mask
maskedI

array([[ 667223,  405901,  304051, ...,  437706,  297139, 1054689],
       [ 305850,  497403,  864256, ...,  379490,  877845, 1021062],
       [  49115,  528221,  317172, ...,  216663,  378081,  784542],
       ...,
       [ 403619,  801594, 1004942, ..., 1045029,  892743,   85734],
       [1004508,  674546,  218285, ...,  513460,  920840,  358819],
       [ 820508,  809818,  672685, ..., -516134, -534883, -306407]])

In [61]:
maskedI.shape

(117703, 100)

In [58]:
import settings
import settings_retrieval

In [69]:
def create_retrieval_submission(I, founds, outfile):
    df = pd.read_csv(os.path.join(settings_retrieval.DATA_DIR, 'index_clean.csv'))
    meta = pd.read_csv(os.path.join(settings.DATA_DIR, 'test', 'test.csv'))  # use same test data as recognition

    labels = [' '.join([str(i) for i in df.iloc[np.extract(x>=0, x)].id.values]) for x in I]

    for i in range(len(labels)):
        if founds[i] == 0:
            labels[i] = ''

    meta['images'] = labels
    meta.to_csv(outfile, index=False, columns=['id', 'images'])

In [70]:
create_retrieval_submission(maskedI, founds, 'sub_retrieval_cut_095.csv')

In [62]:
df_tmp = pd.read_csv(os.path.join(settings_retrieval.DATA_DIR, 'index_clean.csv'))

In [66]:
np.extract(np.array([[-1, 0, 1],[-1, 0, 1]])>=0, [[-1, 0, 1],[-1, 0, 1]])

array([0, 1, 0, 1])

In [27]:
# ransac 
from skimage.measure import ransac
from skimage.transform import warp, AffineTransform

In [41]:
src = np.random.normal(size=(30, 2))
dst = np.random.normal(size=(30, 2)) + np.random.normal(size=(30, 2))
src.shape

(30, 2)

In [42]:

model_robust, inliers = ransac((src, dst), AffineTransform, min_samples=3,
                               residual_threshold=3, max_trials=100)

In [43]:
inliers

array([ True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])